In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorboard.plugins.hparams import api as hp

%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [105]:

df = pd.read_csv('data/only2_big_with_sf.csv')
df.set_index('cell_line', inplace=True)
df.head()

,7105,8813,57147,55732,2268,3075,2519,2729,4800,90529,...,8336,84311,126661,79447,283521,29122,9997,388963,6905,SF
cell_line,,,,,,,,,,,,,,,,,,,,,
NCIH1568-2,5.626439,6.855242,2.185867,4.079805,0.084064,0.176323,6.824513,4.956521,3.766595,2.298658,...,0.622930,5.373996,1.956057,5.482203,0.422233,0.176323,0.918386,1.778209,4.050502,0.607940
SCC9-2,3.964399,5.718088,2.260026,3.451541,0.084064,0.782409,6.039796,5.778734,4.903038,2.711495,...,1.536053,5.042207,2.344828,3.937344,0.000000,0.042644,2.475085,1.176323,4.174726,0.573035
HCC1428-2,4.002703,5.870365,3.621759,3.861955,0.014355,0.485427,4.614121,5.286881,4.429616,3.442280,...,3.033863,4.539159,1.970854,5.895303,0.622930,0.014355,0.918386,2.244887,5.555816,0.450188
U2OS-2,5.098453,6.543496,2.117695,4.115200,0.014355,0.432959,5.684258,4.847997,5.108524,2.189034,...,0.176323,5.028569,2.967169,4.995937,0.111031,0.042644,2.010780,1.925999,4.635754,0.526517
LU99-2,3.665620,6.716717,2.100978,4.176323,0.000000,1.263034,6.124948,4.395063,5.070389,2.669027,...,1.321928,6.613679,1.438293,4.349082,0.000000,0.111031,1.111031,1.632268,5.197315,0.625161


In [106]:

# Create a copy for further processing
dataset = df.copy()
# Get some basic data about dataset
print(len(dataset))

525


In [107]:

# Split the Dataset and create train and test sets
train_dataset = dataset.sample(frac=0.9, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [108]:
########################################################
# Separate labels and features
train_features = train_dataset.drop(["SF"], axis=1)
test_features = test_dataset.drop(["SF"], axis=1)
train_labels = train_dataset["SF"]
test_labels = test_dataset["SF"]

In [109]:
# But we can also apply normalization using sklearn.
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler()
label_scaler = StandardScaler()

In [110]:
# Fit on Training Data
feature_scaler.fit(train_features.values)
label_scaler.fit(train_labels.values.reshape(-1, 1))
########################################################
# Transform both training and testing data
train_features = feature_scaler.transform(train_features.values)
test_features = feature_scaler.transform(test_features.values)
train_labels = label_scaler.transform(train_labels.values.reshape(-1, 1))
test_labels = label_scaler.transform(test_labels.values.reshape(-1, 1))

In [111]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop', 'adagrad', 'adadelta', 'adamax', 'nadam']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [112]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    layers.Dense(1),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='mean_absolute_error',
      metrics=['accuracy'],
  )

  model.fit(train_features, train_labels, epochs=100, callbacks=[tf.keras.callbacks.TensorBoard('data/logs'),  # log metrics
        hp.KerasCallback('data/logs', hparams)]) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(test_features, test_labels)
  return accuracy

In [113]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [114]:
# %tensorboard --logdir logs/hparam_tuning

In [115]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adadelta'}
Epoch 1/100
15/15 [==============================] - 0s 5ms/step - loss: 1.4460 - accuracy: 0.0000e+00
Epoch 2/100
15/15 [==============================] - 0s 4ms/step - loss: 1.4111 - accuracy: 0.0000e+00
Epoch 3/100
15/15 [==============================] - 0s 5ms/step - loss: 1.3401 - accuracy: 0.0000e+00
Epoch 4/100
15/15 [==============================] - 0s 4ms/step - loss: 1.3285 - accuracy: 0.0000e+00
Epoch 5/100
15/15 [==============================] - 0s 4ms/step - loss: 1.3102 - accuracy: 0.0000e+00
Epoch 6/100
15/15 [==============================] - 0s 5ms/step - loss: 1.3224 - accuracy: 0.0000e+00
Epoch 7/100
15/15 [==============================] - 0s 4ms/step - loss: 1.2906 - accuracy: 0.0000e+00
Epoch 8/100
15/15 [==============================] - 0s 3ms/step - loss: 1.3276 - accuracy: 0.0000e+00
Epoch 9/100
15/15 [==============================] - 0s 3ms/step - loss: 1.2610 - accuracy: